# Photo Editing Feature Using TensorFlow

## Step 1: Import Dependencies and VGG19 Model from TensorFlow.Keras
- Import TensorFlow, NumPy, and Matplotlib library
- Import the **VGG19 model** from tensorflow.keras
- Import requests library
- Import **Image** module from PIL library
-Import **BytesIO** module from io library
- Print TensorFlow version


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.applications.vgg19 as vgg19
import requests
from PIL import Image
from io import BytesIO
print(tf.__version__)

## Step 2: Visualize Images with Matplotlib
- Load an image from a given URL and resize it to the specified target size
- Return the loaded image as a numpy array
- Display multiple images in a grid layout
- If only one image is provided, display it in isolation
- Plot a simple line graph using a list of data

In [ ]:
'''Parameters to load image
    url: url of image
    target_size: size of image
   Returns
    Image in form of numpy array
'''
def load_image_from_url(url,target_size=None):
    img_request=requests.get(url)
    img=Image.open(BytesIO(img_request.content))
    if target_size==None:
        return np.array(img)
    else:
        return np.array(img.resize(target_size))

'''Parameters
    images= list of images to plot
    num_rows= number of images in a row (for multiple image plotting)
'''
def plot_image_grid(images,num_rows=1):
    n=len(images)
    if n > 1:
        num_cols=np.ceil(n/num_rows)
        fig,axes=plt.subplots(ncols=int(num_cols),nrows=int(num_rows))
        axes=axes.flatten()
        fig.set_size_inches((15,15))
        for i,image in enumerate(images):
            axes[i].imshow(image)
    else:
        plt.figure(figsize=(10,10))
        plt.imshow(images[0])

'''Parameters
    data= list of data to plot
'''
def plot_graph(data):
    plt.plot(data)
    plt.show()

## Step 3: Provide Style Image and Content Image URL

In [ ]:
style_url="https://assets.catawiki.nl/assets/2019/7/30/f/8/5/f8508825-ec5a-4f5e-bc61-73ff3ded88e2.jpg"
content_url="https://img.bleacherreport.net/img/images/photos/003/825/453/hi-res-ade0501d521ed2716586baa68416bf81_crop_north.jpg?h=533&w=800&q=70&crop_x=center&crop_y=top"

## Step 4: Load and Plot Images from URL


In [ ]:
style_image=load_image_from_url(style_url,target_size=(1024,720))

content_image=load_image_from_url(content_url,target_size=(1024,720))

plot_image_grid([style_image,content_image])

## Step 5: Content and Style Layers Configuration for Neural Network Models
- Define two lists to specify the layers to be used for content and style extraction in a neural network model

In [ ]:
CONTENT_LAYERS=['block4_conv2']

STYLE_LAYERS=['block1_conv1','block2_conv1','block3_conv1', 'block4_conv1', 'block5_conv1']

## Step 6: Create a Model Using Keras Functional API

- Define a function for creating the VGG model with average pooling
- Set the flag to include or exclude the top (fully connected) layers (set to False)
- Specify the pooling type to use (set to **avg** for average pooling) and type of weights to load (set to **imagenet**)
- Freeze the model's weights to prevent training
- Assign the CONTENT_LAYERS list to the variable content_layer
and  STYLE_LAYERS list to the variable style_layer
- Obtain the output tensors of the specified content and style layers in the VGG model
- Return a new Keras model that takes the input of the original VGG model and outputs the selected layers


In [ ]:
def create_vgg_model():
    model=vgg19.VGG19(include_top=False,pooling='avg',weights='imagenet')
    model.trainable=False
    content_layer=CONTENT_LAYERS
    style_layer=STYLE_LAYERS
    output_layers=[model.get_layer(layer).output for layer in (content_layer + style_layer)]
    return tf.keras.models.Model(model.input,output_layers)

## Step 7: Preprocess Content and Style Images Using VGG19
The VGG model requires images to be in the BGR format instead of the RGB format, necessitating preprocessing before use.
- Preprocess the content and style image by expanding its dimensions and applying VGG19-specific preprocessing



In [ ]:
processed_content_image=vgg19.preprocess_input(np.expand_dims(content_image,axis=0))
processed_style_image=vgg19.preprocess_input(np.expand_dims(style_image,axis=0))

## Step 8: Deprocess Function for VGG19-Preprocessed Images
- Define a function to deprocess an image that has been preprocessed with VGG19-specific preprocessing
- Make a copy of the processed image
- If the image has four dimensions, squeeze it to remove the batch dimension
- Check that the image has three dimensions [height, width, and channel]
- Input to deprocess image must be an image of dimension [1, height, width, and channel] or [height, width, and channel]
- Raise a ValueError if the image has an invalid number of dimensions
- Deprocess the image by reversing the VGG19-specific preprocessing steps
- Return the deprocessed image



In [ ]:
'''Parameters
    processed_img= Processed Image in BGR Format
   Returns
    Image with RGB Format
'''

# The deprocess_image function is used to convert an image that has been processed or normalized back into its original form,
# suitable for visualization. This is typically done after neural network operations, such as feature extraction or style transfer,
# where images are often normalized and need to be converted back to their original pixel values.

def deprocess_image(processed_img):
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    assert len(x.shape) == 3, ("Input to deprocess image must be an image of dimension [1, height, width, channel] or [height, width, channel]")
    if len(x.shape) != 3:
        raise ValueError("Invalid input to deprocessing image")

    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

## Step 9: Create Instance of VGG Model

In [ ]:
model=create_vgg_model()

## Step 10: Calculate Content Loss for Neural Style Transfer
- Compute the squared difference between the content representations
- Calculate the mean of the squared differences
-Return the content loss



In [ ]:
'''
 Content Loss= Mean((new_image-base_image)^2)
'''
def get_content_loss(new_image_content,base_image_content):
    return tf.reduce_mean(tf.square(new_image_content-base_image_content))

## Step 11: Calculate Normalized Gram Matrix for Linear Independence
The Gram matrix plays a crucial role in determining linear independence. A collection of vectors is considered linearly independent only if the Gram determinant, that is, the determinant of the Gram matrix, is not equal to zero.

- Get the number of channels in the output
- Reshape the output tensor to have a shape of [-1, channels]
- Compute the Gram matrix by multiplying the reshaped tensor with its transpose
- Get the number of elements in the Gram matrix
- Normalize the Gram matrix by dividing it by the number of elements
- Return the normalized Gram matrix and the number of elements



In [ ]:
'''
 Gram Matrix of a Image is a matrix which we will use to compute correlation between images and will be used in style loss to compute how style of one image
 is similar to other
'''
def get_gram_matrix(output):
    channels=output.shape[-1]
    a=tf.reshape(output,[-1,channels])
    gram_matrix=tf.matmul(a,a,transpose_a=True)
    n=int(gram_matrix.shape[0])
    return gram_matrix/tf.cast(n,'float32'),n

## Step 12: Calculate Style Loss Using Gram Matrices in Neural Style Transfer
- Compute the Gram matrices and heights for the new image and base style
- Ensure that the heights of the Gram matrices are the same
- Get the number of gram features and channels from the new style Gram matrix
- Calculate the style loss using the formula
- Return the style loss



In [ ]:
'''
Style Loss= Mean(1/(2 X nH X nW X nC)^2 X (GramMatrix(style_image)-GramMatrix(new_image))^2)
'''
def get_style_loss(new_image_style,base_style):
    new_style_gram,new_gram_height=get_gram_matrix(new_image_style)
    base_style_gram,base_gram_height=get_gram_matrix(base_style)
    assert new_gram_height==base_gram_height
    gram_features=int(new_style_gram.shape[0])
    gram_channels=int(new_style_gram.shape[-1])
    loss=tf.reduce_mean(tf.square(base_style_gram-new_style_gram)/(2*int(new_gram_height)*(gram_features)*(gram_channels))**2)
    return loss

## Step 13: Calculate Total Loss for Neural Style Transfer with Content and Style Losses

- Separate the style representations of the new image and base style
- Compute the style loss
- Separate the content representations of the new image and base content
- Compute the content loss
- Compute the total loss by combining the content and style losses

In [ ]:
def get_total_loss(new_image_output,base_content_output,base_style_output,alpha=0.001,beta=1):
    new_image_styles=new_image_output[len(CONTENT_LAYERS):]
    base_image_styles=base_style_output[len(CONTENT_LAYERS):]
    style_loss=0
    n=len(new_image_styles)
    for i in range(n):
        style_loss+=get_style_loss(new_image_styles[i],base_image_styles[i])
    new_image_contents=new_image_output[:len(CONTENT_LAYERS)]
    base_image_contents=base_content_output[:len(CONTENT_LAYERS)]
    content_loss=0
    n=len(new_image_contents)
    for i in range(n):
        content_loss+=get_content_loss(new_image_contents[i],base_image_contents[i])/n
    return alpha*content_loss+ beta*style_loss

## Step 14: Extract Output and Generating Image in Neural Style Transfer
- Generated_image is a tensorflow variable which represents new generated image by a network.
- Its value will be changed by training it to minimize the total loss.

In [ ]:
base_style_outputs=model(processed_style_image)
base_content_outputs=model(processed_content_image)

generated_image=tf.Variable(processed_content_image+tf.random.normal(processed_content_image.shape))
optimizer=tf.optimizers.Adam(learning_rate=5.0)

## Step 15: Optimize Generated Image Values Based on Minimizing Total Loss Function
- Create lists to store generated images and store loss values
- Initialize value for best loss
- Initialize the best image with the generated image
- Observe the generated image and optimize its values
- Get the output from the model for the generated image
- Calculate the total loss of the images and append it to the list
- Compute the gradient of the loss with respect to the generated image and apply the gradient update to the generated image
- Clip the image to be in the range of 0-255 and assign the clipped value to the generated image variable
- Store the deprocessed generated image in the list every 10 iterations
- Update the best image if the current loss is better than the previous best loss


In [ ]:
'''
    Parameters
        iterations= number of times to run optimization
    Returns
        best_image : Best Image in all iterations from network
        images: List of Images that are optimized in every iterations this is to track how number of iterations change image appearence
        losses: Total Loss which can be tracked or plot in each iteration
'''
def optimize(iterations):
    images=[]
    losses=[]
    best_loss=20000000
    best_image=generated_image.numpy()
    for i in range(iterations):
        with tf.GradientTape() as tape:
            print("Iteration ",i)
            tape.watch(generated_image) ## Keep Eye on our generated image and optimize its values
            generated_image_content=model(generated_image) ## get output from model for generated image
            loss=get_total_loss(generated_image_content,base_content_outputs,base_style_outputs) ## find total loss of images
            losses.append(loss.numpy())
            gradient=tape.gradient(loss,generated_image)
            optimizer.apply_gradients(zip([gradient],[generated_image]))
            generated_image_clipped=tf.clip_by_value(generated_image, 0, 255) ## Clip image to be in range 0-255
            generated_image.assign(generated_image_clipped) ## assign clipped value of to generated_image variable
            print("LOSS= {0}".format(loss.numpy()))
            if i%10==0:
                images.append(deprocess_image(generated_image.numpy()))
            if loss<best_loss:
                best_image=generated_image.numpy()
                best_loss=loss
    return best_image,images,losses

## Step 16: Optimize Neural Style Transfer and Visualize Loss
- Perform optimization for 10 iterations
- Plot the loss values over iterations


In [ ]:
result,images,losses=optimize(10)
plot_graph(losses)

## Step 17: Plot All Images Obtained in Each Iteration of Optimization

In [ ]:
plot_image_grid(images)

## Step 18: Plot the Final Best Output from the Network
- Set the figure size
- Display the final best output from the network
- Remove the axis labels
- Add a title and show the plot


In [ ]:
plot_image_grid([deprocess_image(result)])

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(deprocess_image(result))

**Observation:**
- The code fetches a style image and a content image from provided URLs.
- It utilizes a VGG19 model to perform neural style transfer, blending the style of the style image with the content of the content image.
- After running optimization for 10 iterations, the code plots the loss, displays intermediate results every 10 iterations, and finally displays the stylized image.